# 6. Data Manipulation IV - Working With Datatypes

The goal of this module is to understand all the different datatypes in `polars`, and learn to construct datatype-specific column expressions that leverage those different datatypes.

But first we import `polars`...

In [25]:
import polars as pl

... and load the data, this time changing the name from `df` to `march_yellow_rides_df` for what's in store in this module.

In [51]:
yellow_rides_column_rename_mapping = {
    "VendorID": "vendor_id",
    "RatecodeID": "ratecode_id",
    "PULocationID": "pu_location_id",
    "DOLocationID": "do_location_id",
    "Airport_fee": "airport_fee",
}
march_yellow_rides_df = (
    pl.read_parquet("../data/yellow_tripdata_2024-03.parquet")
    .rename(yellow_rides_column_rename_mapping)
)

## 5.1. Joining Dataframes

In the previous module, we were interested in seeing some summary statistics about each of the different pickup location IDs:

In [27]:
location_id_summary_df = (
    march_yellow_rides_df
    .group_by("pu_location_id")
    .agg(
        pl.len().alias("count_trips"),
        pl.col("trip_distance").min().name.suffix("_min"),
        pl.col("trip_distance").max().name.suffix("_max"),
        pl.col("trip_distance").mean().name.suffix("_mean"),
    )
    .sort(pl.col("count_trips"), descending=True)
)
location_id_summary_df.head()

pu_location_id,count_trips,trip_distance_min,trip_distance_max,trip_distance_mean
i32,u32,f64,f64,f64
161,163269,0.0,51066.77,2.692728
132,157706,0.0,9211.95,15.76677
237,155631,0.0,44866.77,2.096025
236,146044,0.0,109619.96,3.481146
162,123805,0.0,57408.32,3.030781


This is nice, but if we try to present these results to any sort of business stakeholder, the first thing they'll say is "What are these location IDs? What is their actual name?" Thankfully, that information is stored in another file, `"taxi_zone_lookup.parquet"`. Let's load it in...

In [28]:
zones_df = pl.read_parquet("../data/taxi_zone_lookup.parquet")
zones_df.head()

LocationID,Borough,Zone,service_zone
i32,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""
4,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,"""Staten Island""","""Arden Heights""","""Boro Zone"""


This dataframe contains information about each location ID, and the `str` names for the Borough, Zone, and service zone that each location ID represents. Let's quickly rename the columns to make them snake_case like those of `march_yellow_rides_df`:

In [29]:
zones_df = (
    zones_df
    .rename({
        "LocationID": "location_id",
        "Borough": "borough",
        "Zone": "zone",
    })
)
zones_df.head()

location_id,borough,zone,service_zone
i32,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""
4,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,"""Staten Island""","""Arden Heights""","""Boro Zone"""


Perfect.

If we want to now combine it with the summary statistics we just computed, we can do so by **joining** it into the summary statistics table, using `pl.DataFrame.join()`:

In [30]:
(
    location_id_summary_df
    .join(
        zones_df,
        left_on="pu_location_id",
        right_on="location_id"
    )
    .head()
)

pu_location_id,count_trips,trip_distance_min,trip_distance_max,trip_distance_mean,borough,zone,service_zone
i32,u32,f64,f64,f64,str,str,str
1,372,0.0,35.75,1.34578,"""EWR""","""Newark Airport""","""EWR"""
2,8,0.0,20.23,11.98,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,151,0.0,35.1,7.159073,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""
4,7586,0.0,37.6,2.742518,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,1,0.0,0.0,0.0,"""Staten Island""","""Arden Heights""","""Boro Zone"""


Nice! The data from `zones_df` has been joined into our zone-summary table, and we can see the `"borough"`, `"zone"`, and `"service_zone"` alongside the `"pu_location_id"`! `polars`'s `.join()` functionality has all the same functionality you may be familiar with from Pandas and/or SQL:
1. `location_id_summary_df` specifies the **left** table for the join.
2. `.join(...)` starts the join.
3. `zones_df` specifies the **right** table for the join.
4. `left_on` specifies the column to join on from `location_id_summary_df`.
5. `right_on` specifies the column to join on from `zones_df`.

Just to be clear, though, we didn't have to do the aggregation to `location_id_summary_df`, first; we can join directly into the original `march_yellow_rides_df`:

In [31]:
(
    march_yellow_rides_df
    .select(["tpep_pickup_datetime", "pu_location_id", "do_location_id", "trip_distance", "total_amount"])
    .join(
        zones_df,
        left_on="pu_location_id",
        right_on="location_id",    
    )
#     .select(["tpep_pickup_datetime", "pu_location_id", "do_location_id", "trip_distance", "total_amount", "borough", "zone"])
    .head()
)

tpep_pickup_datetime,pu_location_id,do_location_id,trip_distance,total_amount,borough,zone,service_zone
datetime[ns],i32,i32,f64,f64,str,str,str
2024-03-01 00:18:51,142,239,1.3,16.3,"""Manhattan""","""Lincoln Square East""","""Yellow Zone"""
2024-03-01 00:26:00,238,24,1.1,15.2,"""Manhattan""","""Upper West Side North""","""Yellow Zone"""
2024-03-01 00:09:22,263,75,0.86,10.4,"""Manhattan""","""Yorkville West""","""Yellow Zone"""
2024-03-01 00:33:45,164,162,0.82,14.19,"""Manhattan""","""Midtown South""","""Yellow Zone"""
2024-03-01 00:05:43,263,7,4.9,30.4,"""Manhattan""","""Yorkville West""","""Yellow Zone"""


But now, it's a bit strange that we have the `"pu_location_id"` and `"do_location_id"` in the same dataframe, because it's not clear whether the `"borough"` and `"zone"` refer to the `"pu_location_id"` or the `"do_location_id"`. We can quickly fix that:

In [32]:
zone_columns = ["borough", "zone", "service_zone"]
(
    march_yellow_rides_df
    .select(["tpep_pickup_datetime", "pu_location_id", "do_location_id", "trip_distance", "total_amount"])
    .join(
        zones_df,
        left_on="pu_location_id",
        right_on="location_id",
    )
    .with_columns([pl.col(zone_columns).name.prefix("pu_")])
    .drop(zone_columns)
    .head()
)

tpep_pickup_datetime,pu_location_id,do_location_id,trip_distance,total_amount,pu_borough,pu_zone,pu_service_zone
datetime[ns],i32,i32,f64,f64,str,str,str
2024-03-01 00:18:51,142,239,1.3,16.3,"""Manhattan""","""Lincoln Square East""","""Yellow Zone"""
2024-03-01 00:26:00,238,24,1.1,15.2,"""Manhattan""","""Upper West Side North""","""Yellow Zone"""
2024-03-01 00:09:22,263,75,0.86,10.4,"""Manhattan""","""Yorkville West""","""Yellow Zone"""
2024-03-01 00:33:45,164,162,0.82,14.19,"""Manhattan""","""Midtown South""","""Yellow Zone"""
2024-03-01 00:05:43,263,7,4.9,30.4,"""Manhattan""","""Yorkville West""","""Yellow Zone"""


We can also use the `on` argument in case the name of the two columns is the same in both tables. For example, imagine we wanted to join `zones_df` with the table we created in the prior module, of rides that both started in the same location ID; then we don't need to specify pickup vs dropoff:

In [33]:
(
    march_yellow_rides_df
    .filter(pl.col("do_location_id").eq(pl.col("pu_location_id")))
    .select([
        "tpep_pickup_datetime",
        pl.col("pu_location_id").alias("location_id"),
        "trip_distance",
        "total_amount",
    ])
    .join(
        zones_df,
        on="location_id",
    )
    .head()
)

tpep_pickup_datetime,location_id,trip_distance,total_amount,borough,zone,service_zone
datetime[ns],i32,f64,f64,str,str,str
2024-03-01 00:12:31,158,0.65,15.48,"""Manhattan""","""Meatpacking/West Village West""","""Yellow Zone"""
2024-03-01 00:17:01,158,0.96,15.48,"""Manhattan""","""Meatpacking/West Village West""","""Yellow Zone"""
2024-03-01 01:00:25,143,0.06,8.7,"""Manhattan""","""Lincoln Square West""","""Yellow Zone"""
2024-03-01 00:18:13,236,0.4,11.28,"""Manhattan""","""Upper East Side North""","""Yellow Zone"""
2024-03-01 00:07:35,145,2.53,28.08,"""Queens""","""Long Island City/Hunters Point""","""Boro Zone"""


`polars` also supports all possible joins: anti joins, left joins, right joins, inner joins, outer joins... you've got it all, all you have to use is the `how` argument. To this end, there's a curious difference between `zones_df` and `march_yellow_rides_df`...

In [34]:
print(f"Number of distinct pu_location_id in `march_yellow_rides_df`: {march_yellow_rides_df.select(pl.col('pu_location_id').n_unique())}")
print(f"Number of distinct pu_location_id in `zones_df`: {zones_df.select(pl.col('location_id').n_unique())}")

Number of distinct pu_location_id in `march_yellow_rides_df`: shape: (1, 1)
┌────────────────┐
│ pu_location_id │
│ ---            │
│ u32            │
╞════════════════╡
│ 259            │
└────────────────┘
Number of distinct pu_location_id in `zones_df`: shape: (1, 1)
┌─────────────┐
│ location_id │
│ ---         │
│ u32         │
╞═════════════╡
│ 265         │
└─────────────┘


There are exactly **6** more `location_id`s in `zones_df` than there are `pu_location_id`s in `march_yellow_rides_df`! This makes sense--`zones_df` serves as an index of **all** zones, and there's no guarantee that all zones had at least one ride. What if we wanted to see which zones didn't have any rides that started in them (i.e. `location_id`s with no matching `pu_location_id`)? For that, we can use an anti-join:

In [35]:
(
    zones_df
    .join(march_yellow_rides_df, right_on="pu_location_id", left_on="location_id", how="anti")
)

location_id,borough,zone,service_zone
i32,str,str,str
99,"""Staten Island""","""Freshkills Park""","""Boro Zone"""
103,"""Manhattan""","""Governor's Island/Ellis Island…","""Yellow Zone"""
104,"""Manhattan""","""Governor's Island/Ellis Island…","""Yellow Zone"""
105,"""Manhattan""","""Governor's Island/Ellis Island…","""Yellow Zone"""
110,"""Staten Island""","""Great Kills Park""","""Boro Zone"""
204,"""Staten Island""","""Rossville/Woodrow""","""Boro Zone"""


We can confirm, the result has a length of **6** data points; exactly what we saw in our `n_unique()` check.

And, of course, as with the other Query Statements, we can join not only on a column name `str`, but also on a `pl.Expr` object. Consider the following toy example, where we want to join two columns together based on whether or not they are even or odd:

In [36]:
left_df = pl.DataFrame({"a": [1, 2, 3, 4], "b": ["Danny", "Donna", "Lana", "Sauna"]})
right_df = pl.DataFrame({"c": [1, 2], "d": ["cool", "not cool"]})
display(left_df)
display(right_df)

joined_df = (
    left_df
    .join(
        right_df,
        left_on=pl.col("a") % 2,
        right_on=pl.col("c") % 2
    )
)
display(joined_df)

a,b
i64,str
1,"""Danny"""
2,"""Donna"""
3,"""Lana"""
4,"""Sauna"""


c,d
i64,str
1,"""cool"""
2,"""not cool"""


a,b,d
i64,str,str
1,"""Danny""","""cool"""
0,"""Donna""","""not cool"""
1,"""Lana""","""cool"""
0,"""Sauna""","""not cool"""


Note, `2024-05-30`: At the time of writing, `polars` does not yet support non-equi joins. In other words, the join condition has to be an equality condition, like "dataframe 1 column equals dataframe 2 column". This is less flexible than join options offered by e.g. typical SQL, where you can join on a non-equality condition like "dataframe 1 column is less than dataframe 2 column". But not to fret! `polars` contributors are actively working on this; and you can stay tuned to the progress here: https://github.com/pola-rs/polars/issues/10068.

## 5.2. Concatenating Dataframes

The data we've been working with so far has been the file `"../data/yellow_tripdata_2024-03.parquet"`, which contains all rides given by yellow taxis in NYC in the month of March, 2024. Amazingly, New York City records and publishes this data every month, going all the way back to 2009! What if we wanted to analyze two months' datasets together, in the same dataframe? Well, for that, we have `pl.concat()`.

Let's start by loading in February's yellow taxi trip data. It should have the same schema as the data from March, so we can use the same column name mapping dictionary.

In [63]:
february_yellow_rides_df = (
    pl.read_parquet("../data/yellow_tripdata_2024-02.parquet")
    .rename(yellow_rides_column_rename_mapping)
)
february_yellow_rides_df.head()

vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-02-01 00:04:45,2024-02-01 00:19:58,1,4.39,1,"""N""",68,236,1,20.5,1.0,0.5,1.28,0.0,1.0,26.78,2.5,0.0
2,2024-02-01 00:56:31,2024-02-01 01:10:53,1,7.71,1,"""N""",48,243,1,31.0,1.0,0.5,9.0,0.0,1.0,45.0,2.5,0.0
2,2024-02-01 00:07:50,2024-02-01 00:43:12,2,28.69,2,"""N""",132,261,2,70.0,0.0,0.5,0.0,6.94,1.0,82.69,2.5,1.75
1,2024-02-01 00:01:49,2024-02-01 00:10:47,1,1.1,1,"""N""",161,163,1,9.3,3.5,0.5,2.85,0.0,1.0,17.15,2.5,0.0
1,2024-02-01 00:37:35,2024-02-01 00:51:15,1,2.6,1,"""N""",246,79,2,15.6,3.5,0.5,0.0,0.0,1.0,20.6,2.5,0.0


To combine these two dataframes, we just have to pass them in as a list to `pl.concat()`, and we can see the result:

In [64]:
all_yellow_rides_df = pl.concat([
    february_yellow_rides_df,
    march_yellow_rides_df,
])
display(yellow_rides_df.head())
print(f"{february_yellow_rides_df.shape[0]} rides recorded in `february_yellow_rides_df`.")
print(f"{march_yellow_rides_df.shape[0]} rides recorded in `march_yellow_rides_df`.")
print(f"{all_yellow_rides_df.shape[0]} rides recorded in `all_yellow_rides_df`.")

vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-02-01 00:04:45,2024-02-01 00:19:58,1,4.39,1,"""N""",68,236,1,20.5,1.0,0.5,1.28,0.0,1.0,26.78,2.5,0.0
2,2024-02-01 00:56:31,2024-02-01 01:10:53,1,7.71,1,"""N""",48,243,1,31.0,1.0,0.5,9.0,0.0,1.0,45.0,2.5,0.0
2,2024-02-01 00:07:50,2024-02-01 00:43:12,2,28.69,2,"""N""",132,261,2,70.0,0.0,0.5,0.0,6.94,1.0,82.69,2.5,1.75
1,2024-02-01 00:01:49,2024-02-01 00:10:47,1,1.1,1,"""N""",161,163,1,9.3,3.5,0.5,2.85,0.0,1.0,17.15,2.5,0.0
1,2024-02-01 00:37:35,2024-02-01 00:51:15,1,2.6,1,"""N""",246,79,2,15.6,3.5,0.5,0.0,0.0,1.0,20.6,2.5,0.0


3007526 rides recorded in `february_yellow_rides_df`.
3582628 rides recorded in `march_yellow_rides_df`.
6590154 rides recorded in `all_yellow_rides_df`.


And just like that, we have a combined dataframe of all the yellow taxi rides from February and March! Yellow taxis aren't the only type of taxis in New York City, though; for example, there are also green taxis, which the city of NYC also records. However, a slightly different taxi type means slightly different data... how will it be to combine such data together? Let's see...

Let's start by loading in `march_green_rides_df` from file:

In [75]:
march_green_rides_df = pl.read_parquet("../data/green_tripdata_2024-03.parquet")
march_green_rides_df.head()

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
i32,datetime[ns],datetime[ns],str,i64,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64
2,2024-03-01 00:10:52,2024-03-01 00:26:12,"""N""",1,129,226,1,1.72,12.8,1.0,0.5,3.06,0.0,null,1.0,18.36,1,1,0.0
2,2024-03-01 00:22:21,2024-03-01 00:35:15,"""N""",1,130,218,1,3.25,17.7,1.0,0.5,0.0,0.0,null,1.0,20.2,2,1,0.0
2,2024-03-01 00:45:27,2024-03-01 01:04:32,"""N""",1,255,107,2,4.58,23.3,1.0,0.5,3.5,0.0,null,1.0,32.05,1,1,2.75
1,2024-03-01 00:02:00,2024-03-01 00:23:45,"""N""",1,181,71,1,0.0,22.5,0.0,1.5,0.0,0.0,null,1.0,24.0,1,1,0.0
2,2024-03-01 00:16:45,2024-03-01 00:23:25,"""N""",1,95,135,1,1.15,8.6,1.0,0.5,1.0,0.0,null,1.0,12.1,1,1,0.0


Already, we can see some slight differences between the `all_yellow_rides_df` and `march_green_rides_df`--for example, `march_green_rides_df` has 20 columns, while we're accustomed to the 19 columns in `march_yellow_rides_df`--but there are also a lot of similarities. For one, `march_green_rides_df` has the same `datetime` columns (though with slighly different names), and the same TitleCase ID columns. We'll need to conform the column names so that the correct columns match up with each other upon concatenation.

In [76]:
green_rides_df_column_mapping = {
    "VendorID": "vendor_id",
    "RatecodeID": "ratecode_id",
    "PULocationID": "pu_location_id",
    "DOLocationID": "do_location_id",
    "lpep_pickup_datetime": "tpep_pickup_datetime",
    "lpep_dropoff_datetime": "tpep_dropoff_datetime",
    # "Airport_fee": "airport_fee",  # Doesn't exist in `green_rides_df`
}
march_green_rides_df = march_green_rides_df.rename(green_rides_df_column_mapping)

Great! Now, if we want to finally combine the yellow taxi dataframes and green taxi dataframe with `pl.concat()`, we just pass them in as a list again:

In [77]:
all_rides_df = (
    pl.concat([
        february_yellow_rides_df,
        march_yellow_rides_df,
        march_green_rides_df,
    ])
)

ShapeError: unable to append to a DataFrame of width 19 with a DataFrame of width 20

Even with the renaming, we got an error! That's because, by default, the two dataframes have to have the same schema. This isn't the case in our dataframes, though; however we'd still like to concatenate them! Thankfully, `polars` gives us flexible control over this, with the `how` argument. By default, `how` is set to `how="vertical"`, restricting the concatenation to a strict vertical stacking of the dataframes. However, we can set `how="diagonal"`, and the two dataframes will be concatenated together, with nulls used to fill in the spaces of unshared columns:

In [79]:
all_rides_df = (
    pl.concat(
        [
            february_yellow_rides_df,
            march_yellow_rides_df,
            green_rides_df,
        ],
        how="diagonal"
    )
)
all_rides_df.head()

vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,ehail_fee,trip_type
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2,2024-02-01 00:04:45,2024-02-01 00:19:58,1,4.39,1,"""N""",68,236,1,20.5,1.0,0.5,1.28,0.0,1.0,26.78,2.5,0.0,null,null
2,2024-02-01 00:56:31,2024-02-01 01:10:53,1,7.71,1,"""N""",48,243,1,31.0,1.0,0.5,9.0,0.0,1.0,45.0,2.5,0.0,null,null
2,2024-02-01 00:07:50,2024-02-01 00:43:12,2,28.69,2,"""N""",132,261,2,70.0,0.0,0.5,0.0,6.94,1.0,82.69,2.5,1.75,null,null
1,2024-02-01 00:01:49,2024-02-01 00:10:47,1,1.1,1,"""N""",161,163,1,9.3,3.5,0.5,2.85,0.0,1.0,17.15,2.5,0.0,null,null
1,2024-02-01 00:37:35,2024-02-01 00:51:15,1,2.6,1,"""N""",246,79,2,15.6,3.5,0.5,0.0,0.0,1.0,20.6,2.5,0.0,null,null


Scrolling to the right, we can see that `ehail_fee` and `trip_type` are null, since the yellow trips data doesn't have these two columns--they are brought over from `green_rides_df`.

`pl.concat()` does offer a few more options, such as horizontal concatenation instead of vertical concatenation, but it's not so useful for us here; instead, I leave it to your reading!

# Conclusion

In this module, we learned how to combine multiple dataframes together, using two new Query Statements, `.join()` and `.concat()`.